#### load data

In [1]:
import os
import requests

os.makedirs('local_data', exist_ok=True)

url = "https://arxiv.org/pdf/2303.17760.pdf"
response = requests.get(url)
with open('local_data/camel_paper.pdf', 'wb') as file:
     file.write(response.content)

1. Customized RAG

In [1]:
from camel.embeddings import SentenceTransformerEncoder
from camel.types import EmbeddingModelType

In [4]:
embedding_instance = SentenceTransformerEncoder(model_name="/root/.cache/modelscope/hub/models/BAAI/bge-large-zh-v1.5")

In [6]:
from camel.storages import QdrantStorage

storage_instance = QdrantStorage(
    vector_dim=embedding_instance.get_output_dim(),
    path="local_data",
    collection_name="camel_paper",
)

In [7]:
from camel.retrievers import VectorRetriever

vector_retriever = VectorRetriever(embedding_model=embedding_instance,
                                   storage=storage_instance)

In [8]:
vector_retriever.process(
    content="local_data/camel_paper.pdf",
)

In [9]:
retrieved_info = vector_retriever.query(
    query="To address the challenges of achieving autonomous cooperation, we propose a novel communicative agent framework named role-playing .",
    top_k=1
)
print(retrieved_info)

[{'similarity score': '0.8232748829654635', 'content path': 'local_data/camel_paper.pdf', 'metadata': {'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 10}, 'extra_info': {}, 'text': '6 Conclusion In this paper, we explore the potential of autonomous cooperation among communicative agents and propose a novel cooperative agent framework named role-playing . Our approach enables communicative agents to collaborate autonomously toward completing tasks while requiring minimal human intervention, leading to better solutions are per our thorough evaluations. Through our analysis, we show that achieving autonomous cooperation is challenging due to issues like conversation deviation,'}]


In [11]:
retrieved_info_irrevelant = vector_retriever.query(
    query="If I'm interest in contributing to the CAMEL projec, what should I do?",
    top_k=1,
)

print(retrieved_info_irrevelant)

[{'text': 'No suitable information retrieved from local_data/camel_paper.pdf with similarity_threshold = 0.7.'}]


#### Single agent with RAG

In [ ]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.types import RoleType
from camel.retrievers import AutoRetriever
from camel.types import StorageType

In [ ]:
def single_agent(query: str) -> str:
    # Set agent role
    assistant_sys_msg = """You are a helpful assistant to answer question,
         I will give you the Original Query and Retrieved Context,
        answer the Original Query based on the Retrieved Context,
        if you can't answer the question just say I don't know."""
    
     